<a href="https://colab.research.google.com/github/valefe/playground_transformers/blob/main/Byte_Pair_Encoding_Before_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bpe
!pip install transformers

     |████████████████████████████████| 358kB 4.3MB/s 
     |████████████████████████████████| 21.6MB 87.4MB/s 
     |████████████████████████████████| 747kB 37.2MB/s 
     |████████████████████████████████| 1.8MB 4.3MB/s 
     |████████████████████████████████| 3.2MB 28.9MB/s 
     |████████████████████████████████| 890kB 36.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=c4933c91be59ec9582c1cc779e3c366156b3cb467a786f0e92f0d06534748dc1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from torch import nn, tensor
import pandas as pd
import numpy as np

In [15]:
df = pd.read_csv("description_bisac.csv", nrows=2000, names=["id", "mainDescription", "bisacCode"])
df

,id,mainDescription,bisacCode
0,0,Este livro de Frei Diogo Luís Fuitem faz uma j...,DRA
1,0,Este livro de Frei Diogo Luís Fuitem faz uma j...,DRA
2,1,Nada mais universal do que os assuntos referen...,DRA
3,0,Este livro de Frei Diogo Luís Fuitem faz uma j...,DRA
4,1,Nada mais universal do que os assuntos referen...,DRA
...,...,...,...
1995,42,George e Jane vivem cada um em uma pequena ilh...,HUM
1996,43,"Uma capitania sangrada, oprimida e subjugada p...",HUM
1997,44,80 Desenhos reúne 80 cartuns do artista public...,HUM
1998,45,Menino fala de sua alimentação Um garoto descr...,HUM


# Tokenizing the data

In [4]:
from tokenizers import Tokenizer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE

tokenizer = Tokenizer(BPE())
tokenizer.pre_tokenizer = Whitespace()

In [5]:
from tokenizers.trainers import BpeTrainer

trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer.train(["./description_bisac.csv"], trainer=trainer)

*"Quando você quer alguma coisa, todo o Universo conspira para que você realize seu desejo. De tempos em tempos, surge um livro que muda para sempre a vida de seus leitores. O Alquimista é um deles. Com mais de 65 milhões de exemplares vendidos no mundo inteiro, o mais famoso título de Paulo Coelho já se estabeleceu como um clássico moderno, atemporal e universal. Quase 25 anos após seu lançamento, segue fascinando públicos cada vez maiores, de diferentes gerações. Simples, sábia e inspiradora, esta história refaz os passos de um pastor da Andaluzia que viaja para o deserto egípcio em busca de um tesouro enterrado nas Pirâmides. O que começa como uma jornada para encontrar bens mate¬riais torna-se uma descoberta das riquezas que escondemos dentro de nós mesmos. As belas lições que Santiago aprende pelo caminho nos falam da sabedoria de ouvir o que diz nosso coração, ler os sinais com que deparamos ao longo da vida e, acima de tudo, seguir os nossos sonhos."*

In [7]:
o_alquimista = "Quando você quer alguma coisa, todo o Universo conspira para que você realize seu desejo. De tempos em tempos, surge um livro que muda para sempre a vida de seus leitores. O Alquimista é um deles. Com mais de 65 milhões de exemplares vendidos no mundo inteiro, o mais famoso título de Paulo Coelho já se estabeleceu como um clássico moderno, atemporal e universal. Quase 25 anos após seu lançamento, segue fascinando públicos cada vez maiores, de diferentes gerações. Simples, sábia e inspiradora, esta história refaz os passos de um pastor da Andaluzia que viaja para o deserto egípcio em busca de um tesouro enterrado nas Pirâmides. O que começa como uma jornada para encontrar bens mate¬riais torna-se uma descoberta das riquezas que escondemos dentro de nós mesmos. As belas lições que Santiago aprende pelo caminho nos falam da sabedoria de ouvir o que diz nosso coração, ler os sinais com que deparamos ao longo da vida e, acima de tudo, seguir os nossos sonhos."
encoded = tokenizer.encode(o_alquimista)
encoded.ids[:10]


[2800, 443, 1336, 122, 62, 140, 105, 1993, 11, 860]

In [12]:
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import BertForSequenceClassification  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads

model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased')



Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

In [16]:
from transformers import TextClassificationPipeline

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, framework="pt")